In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import os, shutil
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_dataset_dir = '/content/drive/MyDrive/data/'
base_dir = '/content/drive/MyDrive/data/'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
conv_base = tf.keras.applications.MobileNetV2(
    weights="imagenet",
    input_shape=(224, 224, 3))

14548992/14536120 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))

In [ ]:
# model.summary()

In [ ]:
conv_base.trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch = 10
epochs = 5

train_datagen = ImageDataGenerator(
    rescale = 1. / 255, 
    validation_split = 0.15,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    subset = "training")
label_map = (train_generator.class_indices)

validation_generator = train_datagen.flow_from_directory(
   train_dir,
   target_size = (224, 224),
   batch_size = batch,
   class_mode = 'categorical', 
   subset = "validation", 
   shuffle = False)


label_map

Found 3068 images belonging to 8 classes.
Found 536 images belonging to 8 classes.


{'Засветы': 0,
 'Малая часть упаковки видна на фото': 1,
 'Не читабилен текст': 2,
 'Нормальные': 3,
 'Размытые': 4,
 'Фото не в том режиме': 5,
 'Фото полки где 4-5 товара и видно что фоткали не основ товар': 6,
 'Фото ценника': 7}

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()

In [ ]:
# train_stop = 3604//batch
# test_stop = 401//batch

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [ ]:
%%time
# history = model.fit(train_generator,
#                     steps_per_epoch=train_stop,
#                     epochs=10)

history = model.nm.fit_generator(
   train_generator,
   epochs = epochs,
   validation_data = validation_generator,
   verbose = 1)

AttributeError: ignored

In [ ]:

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)

Found 802 images belonging to 8 classes.


In [ ]:
test = model.evaluate(test_generator, return_dict=True)

81/81 [==============================] - 172s 2s/step - loss: 2.0673 - acc: 0.5087


In [ ]:
model.save('/content/drive/MyDrive/dataset/MobileNetV2.h5')

In [ ]:
df_marks = pd.DataFrame({'x_test': test_generator.filepaths,
     'y_test': test_generator.labels})

In [ ]:
df_marks.to_csv('/content/drive/MyDrive/dataset/file.csv')

In [ ]:
x_test = df_marks.drop(['y_test'],axis=1)
y_test = df_marks['y_test']

In [ ]:
pred = model.predict(test_generator)

print(pred)

[[0.12521413 0.12475312 0.12291747 ... 0.12567626 0.12415649 0.12465773]
 [0.1272174  0.12325341 0.12185048 ... 0.12540701 0.12281095 0.12599932]
 [0.12769473 0.1251578  0.12266451 ... 0.12518789 0.12349916 0.1258472 ]
 ...
 [0.12594518 0.12402366 0.12237936 ... 0.12566645 0.12369394 0.12492571]
 [0.12653293 0.12386465 0.12247828 ... 0.12544173 0.12319683 0.12550466]
 [0.1250222  0.1255292  0.12449571 ... 0.12652539 0.12399682 0.12187402]]


In [ ]:
predicted_classes = np.argmax(pred, axis = 1)

In [ ]:
print(predicted_classes)

[4 3 0 3 3 4 3 3 4 4 3 3 3 3 3 4 4 4 3 3 3 4 3 3 4 3 4 3 3 3 4 3 3 3 3 0 3
 3 3 3 4 3 3 3 4 3 3 3 3 3 3 3 4 4 3 3 3 3 3 4 3 4 3 3 3 3 3 3 3 3 3 3 3 3
 3 4 4 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 4 4 3 3 3 3 3 4 3 0 3 3
 4 4 4 3 3 4 3 4 4 4 3 3 3 4 3 4 3 4 3 3 4 3 3 3 4 3 3 3 4 3 4 3 3 3 4 3 3
 3 0 3 4 3 3 3 3 3 3 4 3 3 3 3 3 4 3 3 3 4 4 3 3 3 4 3 4 3 3 4 4 3 4 3 4 3
 3 3 3 4 3 3 3 4 3 4 4 3 4 4 0 3 3 4 3 4 4 3 4 4 3 3 3 3 3 3 3 3 3 4 4 4 4
 3 3 3 4 5 3 4 3 3 4 3 3 4 3 4 3 4 3 3 4 3 3 3 3 3 4 0 4 4 3 4 3 3 3 4 3 3
 3 4 3 3 4 4 4 3 3 3 3 3 4 4 3 3 4 4 3 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 4 3 3
 4 3 3 3 3 3 3 3 3 4 3 3 4 3 4 4 3 3 3 3 3 3 3 3 4 3 4 3 0 3 3 3 4 4 3 4 3
 4 4 3 4 4 3 3 3 4 3 3 3 3 3 4 4 3 3 3 3 4 3 4 4 4 3 3 3 3 3 3 4 4 4 4 3 3
 3 3 3 5 3 4 3 3 3 4 3 4 4 3 3 4 4 3 4 3 4 3 3 3 4 3 3 4 3 4 3 4 3 3 4 4 4
 3 3 4 0 3 4 3 4 3 3 4 4 4 3 3 3 3 3 4 3 4 3 4 3 3 4 3 3 3 3 3 4 4 3 3 3 3
 4 3 3 3 3 3 3 3 4 4 3 4 4 4 4 3 3 3 4 4 4 4 3 4 4 3 4 3 3 4 4 4 4 4 4 4 3
 3 4 4 3 3 3 4 4 4 3 4 3 

In [ ]:
# df = pd.read_csv("/content/drive/MyDrive/dataset/file.csv")
# x_test = df.drop(['y_test'],axis=1)
# y_test = df['y_test']

In [ ]:
# df_pred = df.join(predicted_classes)
# df_pred

In [ ]:
# df_pred.to_csv('/content/drive/MyDrive/dataset/file_pred.csv')

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
report = classification_report(y_test, predicted_classes)
print(report)

              precision    recall  f1-score   support

           0       0.07      0.25      0.11         4
           1       0.00      0.00      0.00        70
           2       0.00      0.00      0.00        40
           3       0.79      0.62      0.70       650
           4       0.03      0.32      0.05        22
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         8
           7       0.00      0.00      0.00         6

    accuracy                           0.51       802
   macro avg       0.11      0.15      0.11       802
weighted avg       0.64      0.51      0.57       802



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
